In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [2]:
data = pd.read_csv('FinalDataset.csv')

In [3]:
data_std= data.drop('cathode material',axis=1)



## Random Forest For Sp Discharge Cap

In [4]:
from sklearn.model_selection import train_test_split, cross_val_score

y1= data_std['Specific Discharge Capacity at 25 Cycle(mAh g-1)']
x= data_std.drop(['Specific Discharge Capacity at 25 Cycle(mAh g-1)','Estimated Theoretical Capacity','Capacity Retention after 50 Cycles'],axis=1)

#Train Test Split326
x_train_RF_y1,x_test_RF_y1,y_train_RF_y1,y_test_RF_y1= train_test_split(x,y1,test_size=0.2)

In [5]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200], # Number of trees in the forest
    'max_depth': [None, 10, 20], # Maximum depth of the tree
    'min_samples_split': [2, 5, 10], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] # Minimum number of samples required to be at a leaf node
}

# Create a Random Forest regressor
rf = RandomForestRegressor()

# Create a GridSearchCV object with the Random Forest model and hyperparameter grid
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the GridSearchCV object to your training data
grid_search.fit(x_train_RF_y1, y_train_RF_y1)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


In [6]:
#Random Forest Regressor

random_forest_1 = RandomForestRegressor(n_estimators=best_params['n_estimators'],max_depth=best_params['max_depth'], min_samples_split=best_params['min_samples_split'], max_features='auto', random_state=42)
random_forest_1.fit(x_train_RF_y1,y_train_RF_y1)



RandomForestRegressor(max_depth=20, n_estimators=50, random_state=42)

## SVM Regression For Estimated Theoretical Cap

In [7]:
from sklearn.model_selection import train_test_split, cross_val_score

y2= data_std['Estimated Theoretical Capacity']
x= data_std.drop(['Specific Discharge Capacity at 25 Cycle(mAh g-1)','Estimated Theoretical Capacity','Capacity Retention after 50 Cycles'],axis=1)

#Train Test Split326
x_train_SVM_y2,x_test_SVM_y2,y_train_SVM_y2,y_test_SVM_y2= train_test_split(x,y2,test_size=0.2)

In [8]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

# Define the SVM regressor and hyperparameters to tune
svm = SVR()
hyperparams = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.01, 0.1, 1]
}

# Use GridSearchCV to find the best hyperparameters
svm_grid = GridSearchCV(estimator=svm, param_grid=hyperparams, cv=5, scoring='neg_mean_squared_error')
svm_grid.fit(x_train_SVM_y2, y_train_SVM_y2)

best = svm_grid.best_params_

In [9]:
# Create the SVR object
svr = SVR(kernel=best['kernel'], C=best['C'], epsilon=best['epsilon'])

# Train the model
svr.fit(x_train_SVM_y2, y_train_SVM_y2)


SVR(C=10)

# RF for capacity retention

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score

y3= data_std['Capacity Retention after 50 Cycles']
x= data_std.drop(['Specific Discharge Capacity at 25 Cycle(mAh g-1)','Estimated Theoretical Capacity','Capacity Retention after 50 Cycles'],axis=1)

#Train Test Split326

x_train_RF_y3,x_test_RF_y3,y_train_RF_y3,y_test_RF_y3= train_test_split(x,y3,test_size=0.2)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid to search over
param_grid = {
    'n_estimators': [50, 100, 200], # Number of trees in the forest
    'max_depth': [None, 10, 20], # Maximum depth of the tree
    'min_samples_split': [2, 5, 10], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] # Minimum number of samples required to be at a leaf node
}

# Create a Random Forest regressor
rf = RandomForestRegressor()

# Create a GridSearchCV object with the Random Forest model and hyperparameter grid
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)

# Fit the GridSearchCV object to your training data
grid_search.fit(x_train_RF_y3, y_train_RF_y3)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_


In [12]:
#Random Forest Regressor

random_forest_3 = RandomForestRegressor(n_estimators=best_params['n_estimators'],max_depth=best_params['max_depth'], min_samples_split=best_params['min_samples_split'], max_features='auto', random_state=42)
random_forest_3.fit(x_train_RF_y3,y_train_RF_y3)



RandomForestRegressor(min_samples_split=10, random_state=42)

In [13]:
import tkinter as tk
from tkinter import ttk

def calculate():
    # Get input values from GUI widgets
    val1 = float(var1_entry.get())
    val2 = float(var2_entry.get())
    val3 = float(var3_entry.get())
    val4 = float(var4_entry.get())
    val5 = float(var5_entry.get())
    val6 = float(var6_entry.get())

    var= list(zip([val1],[val2],[val3],[val4],[val5],[val6]))
    columns= ['Average Atomic Radius', 'Average Electronegativity',
       'Thermal Conductivity (W/m K)', 'Average Boiling Point',
       'Fusion Heat (KJ/mol)','Average Density']
    x_input= pd.DataFrame(var,columns=columns)
    
    # Perform calculations and update output labels
    out1 = random_forest_1.predict(x_input)
    out2 = svr.predict(x_input)
    out3 = random_forest_3.predict(x_input)
    
    out1_label.config(text="Estimated Specific Discharge Capacity at 25 Cycle(mAh g-1): {:.2f}".format(out1.item()))
    out2_label.config(text="Estimated Theoretical Capacity: {:.2f}".format(out2.item()))
    out3_label.config(text="Capacity Retention after 50 Cycles: {:.2f}".format(out3.item()))
    
# Create the main window and set its properties
root = tk.Tk()
root.title("Cathode Material Predictor For Li_ion Batteries")
root.geometry("1000x900")

# Create input variable labels and entry widgets
var1_label = ttk.Label(root, text="Enter The Value Of Average Atomic Radius")
var1_label.pack()
var1_entry = ttk.Entry(root)
var1_entry.pack()

var2_label = ttk.Label(root, text="Enter The Value Of Average Electronegativity")
var2_label.pack()
var2_entry = ttk.Entry(root)
var2_entry.pack()

var3_label = ttk.Label(root, text="Enter The Value Of Thermal Conductivity (W/m K)")
var3_label.pack()
var3_entry = ttk.Entry(root)
var3_entry.pack()

var4_label = ttk.Label(root, text="Enter The Value Of Average Boiling Point")
var4_label.pack()
var4_entry = ttk.Entry(root)
var4_entry.pack()

var5_label = ttk.Label(root, text="Enter The Value Of Fusion Heat (KJ/mol)")
var5_label.pack()
var5_entry = ttk.Entry(root)
var5_entry.pack()

var6_label = ttk.Label(root, text="Enter The Value Of Average Density")
var6_label.pack()
var6_entry = ttk.Entry(root)
var6_entry.pack()

# Create button to calculate outputs
calc_button = ttk.Button(root, text="Calculate", command=calculate)
calc_button.pack()

# Create output value labels
out1_label = ttk.Label(root, text="Estimated Specific Discharge Capacity at 25 Cycle(mAh g-1): ")
out1_label.pack()

out2_label = ttk.Label(root, text="Estimated Theoretical Capacity: ")
out2_label.pack()

out3_label = ttk.Label(root, text="Capacity Retention after 50 Cycles: ")
out3_label.pack()

# Run the GUI loop
root.mainloop()
